## init environment

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/Users/meruozhu/Downloads/MP_data/MP_codes/MP')
from DPM.task_free_continual_learning.method_provide_model_test import Task_free_continual_learning_provide_model
from DPM.task_free_continual_learning.sampler_provide_model import Sampler
from edbn.Methods.SDL.sdl import transform_data

/Users/meruozhu/opt/anaconda3/envs/pmGrad/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import time
import pandas as pd
import numpy as np
from Data.data import Data
from edbn.Utils.LogFile import LogFile
import edbn.Predictions.setting as setting
from edbn import Methods
from collections import deque
from itertools import islice
from collections import OrderedDict
from PrefixTreeCDD.PrefixTreeClass import PrefixTree
import PrefixTreeCDD.settings as settings
from PrefixTreeCDD.CDD import Window
from skmultiflow.drift_detection import ADWIN, PageHinkley
import math
from numpy import log as ln
import os
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras import utils as ku
import numpy.random as rn

## get model and data

In [6]:
file = '/Users/meruozhu/Downloads/MP_data/MP_codes/MP/Data/Helpdesk.csv'
data = pd.read_csv(file, low_memory=False)
timeformat = "%Y-%m-%d %H:%M:%S"
numEvents = data.shape[0]
print("Num events is {}".format(numEvents))

dataName = 'Helpdesk_mini'
log = LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False)
d = Data(dataName,
            LogFile(filename=file, delim=",", header=0, rows=None, time_attr="completeTime", trace_attr="case",
                    activity_attr='event', convert=False))
d.logfile.keep_attributes(['event', 'role', 'completeTime'])
m = Methods.get_prediction_method("SDL")
s = setting.STANDARD
trainPerc = 0.8
s.train_percentage = trainPerc * 100
# # #
d.prepare(s)
# d.create_batch("normal", timeformat)
is_written = 0

start_time = time.time()
# print(d.train.contextdata)
print("Test Context Data")
print(d.test_orig.contextdata)
basic_model = m.train(d.train,0)

Num events is 21348
PREPARE
CONVERT
PREPROCESSING: Converting event
PREPROCESSING: Converting role
PREPROCESSING: Converting case
K-CONTEXT
Create k-context: 10
SPLIT TRAIN-TEST
Train: 17079
Test: 4269
Test Context Data
      level_0  index  event_Prev9  role_Prev9 completeTime_Prev9  case_Prev9  \
0       17079   4675            0           0                  0         987   
1       17080   7161            0           0                  0        1521   
2       17081   5756            0           0                  0        1221   
3       17082   5757            0           0                  0        1221   
4       17083  14893            0           0                  0        3184   
...       ...    ...          ...         ...                ...         ...   
4264    21343    844            0           0                  0         176   
4265    21344  10513            0           0                  0        2240   
4266    21345  14862            0           0               

## save model

In [7]:
basic_model.save("/Users/meruozhu/Downloads/MP_data/MP_codes/MP/DPM/pretrained_model/Helpdesk")
basic_model_a = tf.keras.models.load_model('/Users/meruozhu/Downloads/MP_data/MP_codes/MP/DPM/pretrained_model/Helpdesk')

2023-05-22 15:16:15.703444: W tensorflow/python/util/util.cc:319] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: /Users/meruozhu/Downloads/MP_data/MP_codes/MP/DPM/pretrained_model/Helpdesk/assets


In [12]:
s = Sampler(data=d)

In [28]:
learning_object=Task_free_continual_learning_provide_model(verbose=False,
                                                        seed=123,
                                                        dev='cpu',
                                                        dim=4,
                                                        hidden_units=100,
                                                        learning_rate=0.005,
                                                        ntasks=2,
                                                        gradient_steps=5,
                                                        loss_window_length=5,
                                                        loss_window_mean_threshold=0.2,
                                                        loss_window_variance_threshold=0.1,                                                         
                                                        MAS_weight=0.5,
                                                        recent_buffer_size=10,
                                                        hard_buffer_size=10,model=basic_model)

In [ ]:
x,y,y_pred = learning_object.method(s, use_hard_buffer=True, continual_learning=True)

In [ ]:
y

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 5, 1, 1, 6, 5, 1, 6, 5])

In [ ]:
y_pred

array([[0.10666234, 0.1920839 , 0.09889975, 0.10217617, 0.10127614,
        0.13955486, 0.15020458, 0.10914224],
       [0.09431007, 0.22994982, 0.09125382, 0.09246794, 0.09079077,
        0.1409911 , 0.16024736, 0.09998909],
       [0.08899624, 0.25372395, 0.08450043, 0.08833944, 0.08509386,
        0.14153828, 0.16340971, 0.09439819],
       [0.08801242, 0.26113018, 0.08252728, 0.08585804, 0.08320002,
        0.1386987 , 0.16921403, 0.09135928],
       [0.08801242, 0.26113018, 0.08252728, 0.08585804, 0.08320002,
        0.1386987 , 0.16921403, 0.09135928],
       [0.08801242, 0.26113018, 0.08252728, 0.08585804, 0.08320002,
        0.1386987 , 0.16921403, 0.09135928],
       [0.08801242, 0.26113018, 0.08252728, 0.08585804, 0.08320002,
        0.1386987 , 0.16921403, 0.09135928],
       [0.08801242, 0.26113018, 0.08252728, 0.08585804, 0.08320002,
        0.1386987 , 0.16921403, 0.09135928],
       [0.08801242, 0.26113018, 0.08252728, 0.08585804, 0.08320002,
        0.1386987 , 0.169214

In [ ]:
y_sup = tf.one_hot(tf.convert_to_tensor(y, dtype=tf.int32), depth=y_pred.shape[1]+5, dtype=tf.float32)

In [ ]:
tf.convert_to_tensor(y, dtype=tf.int32)

<tf.Tensor: id=17009, shape=(20,), dtype=int32, numpy=
array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 6, 5, 1, 1, 6, 5, 1, 6, 5],
      dtype=int32)>

In [ ]:
tf.one_hot(tf.convert_to_tensor(y, dtype=tf.int32), depth=y_pred.shape[1]+5, dtype=tf.float32)

<tf.Tensor: id=17014, shape=(20, 13), dtype=float32, numpy=
array([[0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0

In [ ]:
[np.asarray(row) for row in x.tolist()]

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([6, 1, 1, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 

In [ ]:
print(type(x),x.shape,x[0])

<class 'numpy.ndarray'> (20, 20) [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [ ]:
y_sup = tf.one_hot(tf.convert_to_tensor(y, dtype=tf.int32), depth=y_pred.shape[1], dtype=tf.float32)

TypeError: one_hot() missing 1 required positional argument: 'depth'

## working prediction

In [ ]:
x=xh
print(type(x),x.shape)

NameError: name 'xh' is not defined

In [ ]:
y_pred = basic_model.predict([tf.convert_to_tensor(np.asarray(row)) for row in x.tolist()])
                            #y_pred = self.model(tf.convert_to_tensor(x, dtype=tf.float32))
#y_sup = tf.one_hot(tf.convert_to_tensor(y, dtype=tf.int32), depth=8, dtype=tf.float32)

In [ ]:
lt=[]
for row in xt.tolist():
    #print(np.array(row))
    print(tf.convert_to_tensor(np.array(row)))
    lt.append(tf.convert_to_tensor(np.array(row)))

tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([1 1 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([6 1 1 0 0], shape=(5,), dtype=int64)
tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([6 1 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([0 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)
tf.Tensor([6 1 0 0 0], shape=(5,), dtype=int64)


In [ ]:
xt = x[:, :5]
y_pred = basic_model.predict([tf.convert_to_tensor(np.asarray(row)) for row in xt.tolist()])

### inputs of sdl should be 20 arrays (length can be various)

In [ ]:
log = d.train
inputs, expected, _ = transform_data(log, [a for a in log.attributes() if a != log.time and a != log.trace])
predictions = basic_model.predict(inputs)

In [ ]:
len(inputs)

In [ ]:
[np.asarray(row) for row in x.tolist()]

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([6, 1, 1, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 

In [ ]:
columns = [a for a in log.attributes() if a != log.time and a != log.trace]
num_activities = len(log.values[log.activity]) + 1
print(log.values['event'],log.activity,len(columns) * log.k,len(log.ignoreHistoryAttributes) * log.k,log.k, log.ignoreHistoryAttributes)
col_num_vals = {}
for col in columns:
    if col == log.activity:
        col_num_vals[col] = num_activities
    else:
        col_num_vals[col] = log.contextdata[col].max() + 2
inputs = []
for _ in range(len(columns) * log.k - len(log.ignoreHistoryAttributes) * log.k):
    inputs.append([])
outputs = []
for row in log.contextdata.iterrows():
    # print(row)
    row = row[1]
    # print(row)
    i = 0
    for attr in columns:
        if attr not in log.ignoreHistoryAttributes:
            for k in range(log.k):
                #print(row[attr + "_Prev%i" % k])
                inputs[i].append(row[attr + "_Prev%i" % k])
                print(input[i])
                i += 1
    outputs.append(row[log.activity])

outputs = ku.to_categorical(outputs, num_activities)
for i in range(len(inputs)):
    inputs[i] = np.array(inputs[i])

In [ ]:
col_num_vals

{'event': 8, 'role': 10}

## non working

In [ ]:
print(type(xh),xh.shape)
xh

<class 'numpy.ndarray'> (20, 20)


array([[6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0],
       [6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0],
       [6, 1, 1, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0],
       [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0,

In [ ]:
hb

[{'state': array([6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0]),
  'trgt': 5},
 {'state': array([6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0]),
  'trgt': 5},
 {'state': array([6, 1, 1, 0, 0, 0, 0, 0, 0, 0, 6, 6, 6, 0, 0, 0, 0, 0, 0, 0]),
  'trgt': 5},
 {'state': array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  'trgt': 6},
 {'state': array([1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
  'trgt': 6}]

In [ ]:
xh = np.asarray([_['state'] for _ in hb])
yh = np.asarray([_['trgt'] for _ in hb])

In [ ]:
xh.tolist()
basic_model.predict([np.asarray(row) for row in xh.tolist()])

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 20 array(s), but instead got the following list of 5 arrays: [array([[6],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [6],
       [6],
       [0],
       [0],
       [0],
       [0],
       ...

In [ ]:
yh_pred = basic_model.predict([tf.convert_to_tensor(np.asarray(row)) for row in xh.tolist()])

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 20 array(s), but instead got the following list of 5 arrays: [<tf.Tensor: id=47560, shape=(20, 1), dtype=int64, numpy=
array([[6],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [6],
       [6]...

In [ ]:
np.asarray(test_inputs[0][0].tolist())

AttributeError: 'list' object has no attribute 'tolist'

In [ ]:
[np.asarray(row) for row in x.tolist()][0]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
test_inputs[0]

[array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])]

In [ ]:
basic_model.predict(pr)

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 20 array(s), but instead got the following list of 2 arrays: [array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       ...

In [ ]:
basic_model.predict(test_inputs[0])

ValueError: Error when checking model input: the list of Numpy arrays that you are passing to your model is not the size the model expected. Expected to see 20 array(s), but instead got the following list of 2 arrays: [array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       ...

In [ ]:
basic_model.predict(s.test_inputs)

ValueError: No data provided for "event_Prev0". Need data for each key in: ['event_Prev0', 'event_Prev1', 'event_Prev2', 'event_Prev3', 'event_Prev4', 'event_Prev5', 'event_Prev6', 'event_Prev7', 'event_Prev8', 'event_Prev9', 'role_Prev0', 'role_Prev1', 'role_Prev2', 'role_Prev3', 'role_Prev4', 'role_Prev5', 'role_Prev6', 'role_Prev7', 'role_Prev8', 'role_Prev9']

In [ ]:
with tf.GradientTape() as tape:
    y_pred = basic_model.predict([tf.convert_to_tensor(np.asarray(row)) for row in x.tolist()])
    # print(x)
    # print(tf.convert_to_tensor(x, dtype=tf.float32))
    #y_pred = basic_model(tf.convert_to_tensor(x, dtype=tf.float32))
    #print(y_pred)
    y_sup = tf.one_hot(tf.convert_to_tensor(y, dtype=tf.int32), depth=8, dtype=tf.float32)
    #print(y_sup)
    loss_fn = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.NONE)
    recent_loss = loss_fn(y_sup, y_pred)

In [ ]:
recent_loss

<tf.Tensor: id=5438, shape=(20,), dtype=float32, numpy=
array([0.09371254, 0.08594108, 0.08181993, 0.08013213, 0.08013213,
       0.08013213, 0.08013213, 0.08013213, 0.08013213, 0.08013213,
       0.09324736, 0.09930132, 0.10924446, 0.08013213, 0.08013213,
       0.09733763, 0.11051639, 0.08013213, 0.09733763, 0.11051639],
      dtype=float32)>

## test inputs

In [16]:
test_loss = {i: [] for i in range(2)}
print(test_loss,len(s.test_inputs[0]))

{0: [], 1: []} 60


In [27]:
# for i in range(2):
i=0
#y_pred = self.model.predict([tf.convert_to_tensor(np.asarray(data.test_inputs[i]))])
test_input = np.array([item for item in s.test_inputs[i]])
print(len(test_input))
y_test__pred = basic_model.predict([tf.convert_to_tensor(np.asarray(row)) for row in test_input.T])
#print(test_input)
#y_pred = self.model(tf.convert_to_tensor(np.asarray(test_input), dtype=tf.float32))
y_sup = tf.one_hot(s.test_labels[i], depth=8, dtype=tf.float32)
test_accuracy = np.mean(np.argmax(y_test__pred, axis=1) == s.test_labels[i])
print(test_accuracy)
test_loss[i].append(test_accuracy)
#msg = 'task: {0} step: {1}'.format(t, s)
msg = ' test[{0}]: {1:0.3f}'.format(i, test_accuracy)
print(msg)

60
0.6833333333333333
 test[0]: 0.683


## train the model

In [29]:
x,y = learning_object.method(s, use_hard_buffer=True, continual_learning=True)

recent buffer is full 9


In [30]:
x

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [37]:
with tf.GradientTape() as tape:
    y_pred = basic_model([np.asarray(row).reshape(-1,1) for row in x.T.tolist()],training=True)

In [34]:
[tf.convert_to_tensor(np.asarray(row)) for row in x.T.tolist()]

[<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 1, 0, 1, 0, 0])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0, 6, 0, 6, 0, 0])>,
 <tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 0, 0

In [32]:
y_pred

<tf.Tensor: shape=(10, 8), dtype=float32, numpy=
array([[4.5483053e-04, 9.8203170e-01, 1.8678904e-04, 4.0509077e-04,
        1.0185934e-02, 9.0766710e-04, 5.0124726e-03, 8.1559614e-04],
       [3.9354252e-04, 9.8614991e-01, 1.5926590e-04, 2.0725017e-04,
        8.1078336e-03, 6.1356381e-04, 3.9118109e-03, 4.5686855e-04],
       [3.6520243e-04, 9.8218095e-01, 1.6232373e-04, 4.7122428e-04,
        9.0080965e-03, 1.1098512e-03, 5.7847188e-03, 9.1768074e-04],
       [2.2469477e-04, 9.8566103e-01, 1.2445789e-04, 3.3268597e-04,
        6.7049712e-03, 6.9527118e-04, 5.6386483e-03, 6.1825226e-04],
       [4.7505356e-04, 9.8479074e-01, 1.8243995e-04, 3.8534755e-04,
        8.1527131e-03, 8.5320248e-04, 4.3781293e-03, 7.8239699e-04],
       [1.7853804e-04, 1.4891672e-01, 1.3276840e-03, 5.7457999e-04,
        1.3633753e-03, 6.7535497e-04, 8.4436572e-01, 2.5980219e-03],
       [6.2064629e-04, 9.8111492e-01, 2.2992709e-04, 3.2968863e-04,
        1.0972153e-02, 8.9997175e-04, 5.2310731e-03, 6.016473